In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

In [2]:
# Import objects from pyomo package
from pyomo.environ import ConcreteModel, SolverFactory, units as pyunits

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import Flash unit model from idaes.models.unit_models
from idaes.models.unit_models import Flash

In [3]:
import pyomo.contrib.parmest.parmest as parmest

In [4]:
from idaes.models.properties.modular_properties.examples.CO2_bmimPF6_PR import (
    configuration,
)

from idaes.models.properties.modular_properties import GenericParameterBlock

In [5]:
import pandas as pd

# Load data from csv
data = pd.read_csv("CO2_IL_298.csv")

In [6]:
def PR_model(data):

    m = ConcreteModel()

    m.fs = FlowsheetBlock(dynamic=False)

    m.fs.properties = GenericParameterBlock(**configuration)

    m.fs.state_block = m.fs.properties.build_state_block([1], defined_state=True)

    m.fs.state_block[1].flow_mol.fix(1)
    x = float(data["x_carbon_dioxide"]) + 0.5
    m.fs.state_block[1].temperature.fix(float(data["temperature"]))
    m.fs.state_block[1].pressure.fix(float(data["pressure"]))
    m.fs.state_block[1].mole_frac_comp["bmimPF6"].fix(1 - x)
    m.fs.state_block[1].mole_frac_comp["carbon_dioxide"].fix(x)

    # parameter - kappa_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa["bmimPF6", "bmimPF6"].fix(0)
    m.fs.properties.PR_kappa["bmimPF6", "carbon_dioxide"].fix(-0.047)
    m.fs.properties.PR_kappa["carbon_dioxide", "carbon_dioxide"].fix(0)
    m.fs.properties.PR_kappa["carbon_dioxide", "bmimPF6"].fix(0.002)

    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.INFO)

    # Fix the state variables on the state block
    m.fs.state_block[1].pressure.unfix()
    m.fs.state_block[1].temperature.fix(float(data["temperature"]))
    m.fs.state_block[1].mole_frac_phase_comp["Liq", "bmimPF6"].fix(
        float(data["x_bmimPF6"])
    )
    m.fs.state_block[1].mole_frac_phase_comp["Liq", "carbon_dioxide"].fix(
        float(data["x_carbon_dioxide"])
    )
    m.fs.state_block[1].mole_frac_comp["bmimPF6"].fix(float(data["x_bmimPF6"]))
    m.fs.state_block[1].mole_frac_comp["carbon_dioxide"].unfix()
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa["bmimPF6", "carbon_dioxide"].setlb(-5)
    m.fs.properties.PR_kappa["bmimPF6", "carbon_dioxide"].setub(5)

    m.fs.properties.PR_kappa["carbon_dioxide", "bmimPF6"].setlb(-5)
    m.fs.properties.PR_kappa["carbon_dioxide", "bmimPF6"].setub(5)

    # Return initialized flash model
    return m

In [7]:
from idaes.core.util.model_statistics import degrees_of_freedom
import pytest

test_data = {
    "temperature": 298,
    "pressure": 812323,
    "x_bmimPF6": 0.86,
    "x_carbon_dioxide": 0.14,
}

m = PR_model(test_data)

# Check that degrees of freedom is 0
assert degrees_of_freedom(m) == 0

2023-02-18 12:04:34 [INFO] idaes.init.fs.state_block: Starting initialization


WARNING (W1002): Setting Var
    'fs.state_block[1].log_mole_frac_tbub[Vap,Liq,carbon_dioxide]' to a
    numeric value `1.3789905650578088e-06` outside the bounds (None, 0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002


2023-02-18 12:04:34 [INFO] idaes.init.fs.state_block: Dew and bubble point initialization: optimal - Optimal Solution Found.


2023-02-18 12:04:34 [INFO] idaes.init.fs.state_block: Equilibrium temperature initialization completed.


2023-02-18 12:04:34 [INFO] idaes.init.fs.state_block: State variable initialization completed.


2023-02-18 12:04:35 [INFO] idaes.init.fs.state_block: Phase equilibrium initialization: optimal - Optimal Solution Found.


2023-02-18 12:04:35 [INFO] idaes.init.fs.state_block: Property initialization: optimal - Optimal Solution Found.


2023-02-18 12:04:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.


In [8]:
variable_name = [
    "fs.properties.PR_kappa['bmimPF6', 'carbon_dioxide']",
    "fs.properties.PR_kappa['carbon_dioxide', 'bmimPF6']",
]

In [9]:
def SSE(m, data):
    expr = (float(data["pressure"]) - m.fs.state_block[1].pressure) ** 2
    return expr * 1e-7

In [10]:
pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

obj_value, parameters = pest.theta_est()

2023-02-18 12:04:35 [INFO] idaes.init.fs.state_block: Starting initialization


WARNING (W1002): Setting Var
    'fs.state_block[1].log_mole_frac_tbub[Vap,Liq,carbon_dioxide]' to a
    numeric value `4.301303339264284e-06` outside the bounds (None, 0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002


2023-02-18 12:04:35 [INFO] idaes.init.fs.state_block: Dew and bubble point initialization: optimal - Optimal Solution Found.


2023-02-18 12:04:35 [INFO] idaes.init.fs.state_block: Equilibrium temperature initialization completed.


2023-02-18 12:04:35 [INFO] idaes.init.fs.state_block: State variable initialization completed.


...

2023-02-18 12:04:46 [INFO] idaes.init.fs.state_block: State variable initialization completed.


2023-02-18 12:04:46 [INFO] idaes.init.fs.state_block: Phase equilibrium initialization: optimal - Optimal Solution Found.


2023-02-18 12:04:46 [INFO] idaes.init.fs.state_block: Property initialization: optimal - Optimal Solution Found.


2023-02-18 12:04:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.


Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [11]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k, v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 29.282891

The values for the parameters are as follows:
fs.properties.PR_kappa[bmimPF6,carbon_dioxide] = -0.4071428400296551
fs.properties.PR_kappa[carbon_dioxide,bmimPF6] = 0.020593684002515204
